In [0]:
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression

In [0]:
spark = SparkSession.builder.appName('b').getOrCreate()

In [0]:
df=spark.read.csv('/FileStore/tables/cruise_ship_info.csv',header=True,inferSchema=True)

In [0]:
df.columns

Out[35]: ['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew']

In [0]:
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol="Cruise_line", outputCol="Cruise_line_cat")
final_df = indexer.fit(df).transform(df)
final_df.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+---------------+
 Ship_name|Cruise_line|Age| Tonnage|passengers|length|cabins|passenger_density|crew|Cruise_line_cat|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+---------------+
 Journey| Azamara| 6|30.276999999999997| 6.94| 5.94| 3.55| 42.64|3.55| 16.0|
 Quest| Azamara| 6|30.276999999999997| 6.94| 5.94| 3.55| 42.64|3.55| 16.0|
Celebration| Carnival| 26| 47.262| 14.86| 7.22| 7.43| 31.8| 6.7| 1.0|
 Conquest| Carnival| 11| 110.0| 29.74| 9.53| 14.88| 36.99|19.1| 1.0|
 Destiny| Carnival| 17| 101.353| 26.42| 8.92| 13.21| 38.36|10.0| 1.0|
 Ecstasy| Carnival| 22| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
 Elation| Carnival| 15| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
 Fantasy| Carnival| 23| 70.367| 20.56| 8.55| 10.22| 34.23| 9.2| 1.0|
Fascination| Carnival| 19| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
 Freedom| Carnival| 6|110.23899999999999| 37.0| 9.51| 14.87| 29.79|11.5| 1.0|
 Glory| Carnival| 10| 110.0| 29.74| 9.51| 14.87| 36.99|11.6| 1.0|
 Holiday| Carnival| 28| 46.052| 14.52| 7.27| 7.26| 31.72| 6.6| 1.0|
Imagination| Carnival| 18| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
Inspiration| Carnival| 17| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
 Legend| Carnival| 11| 86.0| 21.24| 9.63| 10.62| 40.49| 9.3| 1.0|
 Liberty*| Carnival| 8| 110.0| 29.74| 9.51| 14.87| 36.99|11.6| 1.0|
 Miracle| Carnival| 9| 88.5| 21.24| 9.63| 10.62| 41.67|10.3| 1.0|
 Paradise| Carnival| 15| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
 Pride| Carnival| 12| 88.5| 21.24| 9.63| 11.62| 41.67| 9.3| 1.0|
 Sensation| Carnival| 20| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+---------------+
only showing top 20 rows

In [0]:
assembler = VectorAssembler( inputCols=['Cruise_line_cat',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density'],outputCol='features')

In [0]:
output = assembler.transform(final_df)

In [0]:
output.head(1)

Out[43]: [Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, Cruise_line_cat=16.0, features=DenseVector([16.0, 6.0, 30.277, 6.94, 5.94, 3.55, 42.64]))]

In [0]:
final_data = output.select('features','crew')

In [0]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [0]:
lr= LinearRegression(labelCol='crew')

In [0]:
lr_model = lr.fit(train_data)

In [0]:
test_result = lr_model.evaluate(test_data)

In [0]:
test_result.rootMeanSquaredError

Out[56]: 1.2296313308944824

In [0]:
test_result.r2

Out[57]: 0.910502905786292

In [0]:
unlabeled_data = test_data.select('features')

In [0]:
predictions = lr_model.transform(unlabeled_data)

In [0]:
from pyspark.sql.functions import corr

In [0]:
df.select(corr('crew','cabins')).show()

+------------------+
corr(crew, cabins)|
+------------------+
0.9508226063578497|
+------------------+